Load libraries and read data into dataframes

In [3]:
import pandas as pd

In [4]:
ppd = pd.read_csv("ppd_complaints.csv")
ppd_disc = pd.read_csv("ppd_complaint_disciplines.csv")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 360: invalid start byte

In [4]:
ppd.head()

,cap_number,date_received,dist_occurrence,general_cap_classification,summary
0,17-0001,2017-01-12 00:00:00,25,UNPROFESSIONAL CONDUCT,"The complainant, CN, 59/H/F,states that she wa..."
1,17-0002,2017-01-13 00:00:00,18,LACK OF SERVICE,"The complainant, DH, 28/B/M, states that he di..."
2,17-0003,2017-01-12 00:00:00,7,PHYSICAL ABUSE,"The complainant, AK, 39/B/M, states that he wa..."
3,17-0004,2017-01-12 00:00:00,3,LACK OF SERVICE,"The complainant, EG, stated that she did not r..."
4,17-0005,2017-01-12 00:00:00,35,DOMESTIC,"The complainant, IV, 41/B/F, states her ex-hus..."


Overall, the amount of data is pretty small. Therefore we perform an outer join to the two tables to preserve the summary, classification, etc. We may remove rows later, but for now it's fine.

In [5]:
ppd = ppd.merge(ppd_disc, how='outer', on="cap_number")

In [6]:
ppd.head()

,cap_number,date_received,dist_occurrence,general_cap_classification,summary,po_initials,po_race,po_sex,allegations_investigated,investigative_findings,disciplinary_findings
0,17-0001,2017-01-12 00:00:00,25,UNPROFESSIONAL CONDUCT,"The complainant, CN, 59/H/F,states that she wa...",CV,White,Male,Unprofessional Conduct,Other than Sustained Finding,Not Applicable
1,17-0001,2017-01-12 00:00:00,25,UNPROFESSIONAL CONDUCT,"The complainant, CN, 59/H/F,states that she wa...",CV,White,Male,Criminal Allegation,Other than Sustained Finding,Not Applicable
2,17-0002,2017-01-13 00:00:00,18,LACK OF SERVICE,"The complainant, DH, 28/B/M, states that he di...",RB,Black,Female,Lack of Service,Sustained Finding,Training/Counseling
3,17-0002,2017-01-13 00:00:00,18,LACK OF SERVICE,"The complainant, DH, 28/B/M, states that he di...",RF,Latino,Male,Lack of Service,Sustained Finding,Training/Counseling
4,17-0002,2017-01-13 00:00:00,18,LACK OF SERVICE,"The complainant, DH, 28/B/M, states that he di...",KB,Black,Male,Lack of Service,Other than Sustained Finding,Not Applicable


# Exploratory analysis

Print out frequency summaries of various columns in the dataset  

In [7]:
ppd["investigative_findings"].value_counts()

Pending                         401
Other than Sustained Finding    396
Sustained Finding                69
Name: investigative_findings, dtype: int64

In [8]:
ppd["disciplinary_findings"].value_counts()

Not Applicable         428
Pending                406
Training/Counseling     27
Guilty Finding           5
Name: disciplinary_findings, dtype: int64

In [9]:
ppd["allegations_investigated"].value_counts()

Investigation OnGoing          401
Departmental Violation         138
Lack of Service                124
Verbal Abuse                    60
Physical Abuse                  59
Unprofessional Conduct          30
Criminal Allegation             21
Civil Rights Complaint          11
Harassment                      10
Domestic                         5
Other Misconduct                 3
Sexual Crime/Misconduct          2
Disciplinary Code Violation      2
Name: allegations_investigated, dtype: int64

In [10]:
ppd["general_cap_classification"].value_counts()

 DEPARTMENTAL VIOLATIONS       223
 LACK OF SERVICE               221
 PHYSICAL ABUSE                161
 VERBAL ABUSE                   66
 CRIMINAL ALLEGATION            51
 NON-INVESTIGATORY INCIDENT     45
 UNPROFESSIONAL CONDUCT         32
 HARASSMENT                     24
 CIVIL RIGHTS COMPLAINT         21
 DOMESTIC                       11
 SEXUAL CRIME/MISCONDUCT        10
 DRUGS                           2
Name: general_cap_classification, dtype: int64

By far the most common districts for complaints is the 25th district. Some districts may not have any complaints (becuase location is unknown for some). Also, some labels seem to be messed up ('5-') or missing. '77' is the code for the airport.

In [11]:
ppd["dist_occurrence"].value_counts()

25     98
18     73
14     72
19     61
15     58
24     56
35     54
12     49
22     48
16     42
8      40
2      35
39     30
9      29
6      25
17     23
3      15
26     15
5      14
7      11
1      10
UNK     6
5-      2
77      1
Name: dist_occurrence, dtype: int64

# Analysis of guilty findings

In [12]:
guiltyfindings = ppd[ppd["disciplinary_findings"] == "Guilty Finding"]
print(guiltyfindings)

    cap_number        date_received dist_occurrence  \
88     17-0049  2017-02-06 00:00:00              16   
89     17-0049  2017-02-06 00:00:00              16   
214    17-0106  2017-03-10 00:00:00              22   
215    17-0106  2017-03-10 00:00:00              22   
216    17-0106  2017-03-10 00:00:00              22   

    general_cap_classification  \
88                  HARASSMENT   
89                  HARASSMENT   
214                 HARASSMENT   
215                 HARASSMENT   
216                 HARASSMENT   

                                               summary po_initials po_race  \
88   The complainant, GF, 24/F, states that Officer...          AP   Black   
89   The complainant, GF, 24/F, states that Officer...          AP   Black   
214  The complainant, J.S., 28/B/F, states that Off...          TD   Black   
215  The complainant, J.S., 28/B/F, states that Off...          TD   Black   
216  The complainant, J.S., 28/B/F, states that Off...          TD   Black

There's only two separate cases here, so let's print out the summaries.

In [13]:
print(guiltyfindings["summary"][88])

The complainant, GF, 24/F, states that Officer AP, conducted himself inappropriately.  According to the complainant, she called 911 to report an assault; Officer AP responded.  At approximately 3:30 PM, she received a text from Officer AP requesting that she call him; he was just checking to see if she was okay.  The complainant called him at approximately 4 PM and left a message on his voicemail saying that she was safe.  She then received another text message from Officer AP stating that he needed to call back and ask her something.  Officer AP next called the complainant at approximately 6 PM and asked her to call him directly and not utilize 911.  He then asked the complainant to go out for dinner; the complainant declined the offer.  Officer AP then made several other comments and suggestions to the complainant that she felt were highly inappropriate.  The complainant ended up blocking the officer's number; she states that during their conversations, Officer AP did not speak of an

In [14]:
print(guiltyfindings["summary"][214])

The complainant, J.S., 28/B/F, states that Officer T.D. inappropriately utilized social media. According to the complaint, her vehicle was vandalized at her residence and she reported the incident to police. Three days later, she was on Facebook, and came across photos of her vandalized vehicle on Officer T.D.'s personal Facebook page. The complainant maintains that Officer T.D. posted specifics about what had been done to her vehicle, included information obtained from another officer about the incident, and openly joked about the incident in his social media postings.  


Some things to note: the 5 guilty findings were for 2 different cases, with one case having two violations and the other having three. Both fell into the classification of harrassment. Both POs were male, their victims female. Also in both cases, there was recoverable circumstantial evidence in the form of text messages and/or facebook posts.

# Analysis of training/counseling findings 

In [15]:
tcfindings = ppd[ppd["disciplinary_findings"] == "Training/Counseling"]
print(tcfindings.head())

   cap_number        date_received dist_occurrence general_cap_classification  \
2     17-0002  2017-01-13 00:00:00              18            LACK OF SERVICE   
3     17-0002  2017-01-13 00:00:00              18            LACK OF SERVICE   
31    17-0019  2017-01-20 00:00:00              25               VERBAL ABUSE   
32    17-0019  2017-01-20 00:00:00              25               VERBAL ABUSE   
84    17-0047  2017-02-02 00:00:00              19    DEPARTMENTAL VIOLATIONS   

                                              summary po_initials po_race  \
2   The complainant, DH, 28/B/M, states that he di...          RB   Black   
3   The complainant, DH, 28/B/M, states that he di...          RF  Latino   
31  The complainant, ET, 42/B/M, states that he wa...          RO   Black   
32  The complainant, ET, 42/B/M, states that he wa...          RO   Black   
84  The complainant, LF, 55/B/M, states that he is...          VG   Black   

    po_sex allegations_investigated investigative_

In [16]:
tcfindings["cap_number"].value_counts()

17-0085    3
17-0055    2
17-0103    2
17-0105    2
17-0019    2
17-0002    2
17-0047    2
17-0098    1
17-0050    1
17-0175    1
17-0172    1
17-0125    1
17-0422    1
17-0265    1
17-0143    1
17-0104    1
17-0058    1
17-0200    1
17-0086    1
Name: cap_number, dtype: int64

Eight cases have multiple sustained findings. Overall, slightly more than half of the sustained findings belong to cases with multiple findings (15: > 1 finding, 12: 1 finding).

In [17]:
tcfindings["general_cap_classification"].value_counts()

 DEPARTMENTAL VIOLATIONS    12
 VERBAL ABUSE                4
 LACK OF SERVICE             3
 DOMESTIC                    3
 PHYSICAL ABUSE              2
 UNPROFESSIONAL CONDUCT      2
 SEXUAL CRIME/MISCONDUCT     1
Name: general_cap_classification, dtype: int64

In [18]:
tcfindings["allegations_investigated"].value_counts()

Departmental Violation    24
Lack of Service            2
Domestic                   1
Name: allegations_investigated, dtype: int64

In [19]:
tcfindings["po_sex"].value_counts()

Male      18
Female     9
Name: po_sex, dtype: int64

In [20]:
tcfindings["dist_occurrence"].value_counts()

18    5
25    4
35    4
19    4
16    2
12    2
6     2
24    1
7     1
15    1
14    1
Name: dist_occurrence, dtype: int64

# By officer

Although the data is semi-anonymized, it's still possible to see if there are any spikes in the number of complaints that come from specific officers.

In [21]:
ppd["po_id"] = ppd[["po_initials","po_race","po_sex","dist_occurrence"]].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [22]:
ppd["po_id"].value_counts().head(25)

UNK_UNK_UNK_15        8
UNK_UNK_UNK_25        8
UNK_UNK_UNK_18        7
UNK_UNK_UNK_19        7
KC_White_Male_24      6
FB_White_Male_14      6
KF_White_Male_22      5
MK_White_Male_24      5
UNK_UNK_UNK_24        5
EM_White_Male_24      4
FB_White_Male_8       4
MS_White_Male_24      4
CJ_Black_Male_14      4
AC_Black_Male_25      4
ER_Black_Male_18      4
RM_White_Male_18      4
PS_White_Male_25      4
UNK_UNK_UNK_9         4
JO_White_Male_25      4
JH_Black_Female_19    4
UNK_UNK_UNK_35        4
RW_White_Male_39      4
RH_White_Male_2       3
MC_White_Male_22      3
S_Black_Female_1      3
Name: po_id, dtype: int64

In [24]:
ppd[~ppd["po_id"].isin(["UNK_UNK_UNK_15"])].value_counts().head(25)

AttributeError: 'DataFrame' object has no attribute 'value_counts'

Many of the "most commonly offending" cops were unknown, and not likely to be the same person (or possibly multiple people). Besides that, several police officers had multiple complaints, as many as 6 that were processed or pending. (This count includes pending and resolved cases)